## Notebook match and combine dataframes with the enrollment data

### going to use fuzzy matching to acomplish the task

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### File Locations
# File location and type

# 2022 Enrollment Data
enroll_df = pd.read_excel('data\collected.xlsx', sheet_name='2022_enrollment')

# Sheet with manually collected data
manual_df = pd.read_excel('data\collected.xlsx', sheet_name='manual')

# Sheet with calculated data
area_sqft_df = pd.read_csv('fields_notebook_export_w_geos.csv')

high_school_df = manual_df[manual_df['Level'] == 'high school']
# high_school_df.head()
# enroll_df.head()
# area_sqft_df.head()

##### Working to Load all the data
enroll_df.head()



,school_id,school_name,enrollment_total,enrollment_classification,division,level
0,9448,Macomb - Dakota HS,2876,2876,A,high_school
1,2237,Grand Blanc HS,2812,2812,A,high_school
2,5792,East Kentwood HS,2678,2678,A,high_school
3,6980,Dearborn - Fordson HS,2613,2613,A,high_school
4,3020,Shelby Township - Utica Eisenhower HS,2555,2555,A,high_school


In [6]:
### Lookup school_id and school_name from enroll_df and copy it into the area_sqft_df

import difflib
area_sqft_df['school_name'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name'], n=1, cutoff=0.6))
area_sqft_df['school_name'] = area_sqft_df['school_name'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
new_df = enroll_df.merge(area_sqft_df, on='school_name', how='left')

# # drop rows with null values from new_df
# new_df = new_df.dropna(subset=['home_plate'])

area_sqft_df.head()
area_sqft_df.info()
enroll_df.head()
new_df.info()
new_df.head()

## export to check
new_df.to_csv('test_merge_new_df.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   222 non-null    int64  
 1   field        222 non-null    object 
 2   home_plate   222 non-null    object 
 3   foul_ft      222 non-null    float64
 4   fop_ft       222 non-null    float64
 5   foul_pct     222 non-null    float64
 6   school_name  159 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 12.3+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 772 entries, 0 to 771
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   school_id                  772 non-null    int64  
 1   school_name                772 non-null    object 
 2   enrollment_total           772 non-null    int64  
 3   enrollment_classification  772 non-null    int64  
 4   division 

In [ ]:
### Lookup enrollment numbers and school id from the enrollment sheet and add them to the high school sheet

import difflib

high_school_df['match'] = high_school_df['School'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name'].tolist(), n=1, cutoff=0.6))
high_school_df['match'] = high_school_df['match'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
high_school_df['enrollment'] = high_school_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['enrollment_classification'].values[0] if type(x) == str else np.nan)
high_school_df['school_id'] = high_school_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['school_id'].values[0] if type(x) == str else np.nan)
high_school_df['Div'] = high_school_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['division'].values[0] if type(x) == str else np.nan)
high_school_df['enrollment'] = high_school_df['enrollment'].fillna(0)
high_school_df['school_id'] = high_school_df['school_id'].fillna(0)
high_school_df = high_school_df.drop(columns=['match'])
high_school_df.head()

high_school_df.describe()

# Export csv to check output

high_school_df.to_csv('data\high_school_df_test.csv')



In [ ]:
## Lookup enrollment numbers and school id from the enrollment sheet and add them to the area_sqft sheet

import difflib

area_sqft_df['match'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name'].tolist(), n=1, cutoff=0.8))
area_sqft_df['match'] = area_sqft_df['match'].apply(lambda x: x[0] if len(x) > 0 else np.nan)
area_sqft_df['enrollment'] = area_sqft_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['enrollment_classification'].values[0] if type(x) == str else np.nan)
area_sqft_df['school_id'] = area_sqft_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['school_id'].values[0] if type(x) == str else np.nan)
area_sqft_df['Div'] = area_sqft_df['match'].apply(lambda x: enroll_df[enroll_df['school_name'] == x]['division'].values[0] if type(x) == str else np.nan)
area_sqft_df['enrollment'] = area_sqft_df['enrollment'].fillna(0)
area_sqft_df['school_id'] = area_sqft_df['school_id'].fillna(0)
area_sqft_df = area_sqft_df.drop(columns=['match'])
area_sqft_df.info()

## Export csv to check output
area_sqft_df.to_csv('area_sqft_df_test.csv')



## Code above works

### the lookups need to be a lot better. only matched 64 records out of 222


#### the problem might just be how messy the field names are in the area_sqft_df
#### might need to clean them up at the google maps level

In [ ]:
##### THIS BLOCK IS KIND OF A MESS AND PROBABLY ISN'T GOING TO BE USED

### merge the dataframes
import difflib
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process


high_school_df['match'] = high_school_df['School'].str.lower()
enroll_df['match'] = enroll_df['school_name'].str.lower()
area_sqft_df['match'] = area_sqft_df['field'].str.lower()

## Drop the columns that are not needed
high_school_df = high_school_df.drop(columns=['FOP_ratio','Foul-ratio','Foul_pct_of_whole'])


## Match the school names
high_school_df['mat2'] = high_school_df['match'].apply(lambda x: difflib.get_close_matches(x, enroll_df['match'], n=1, cutoff=0.8))
area_sqft_df['mat2'] = area_sqft_df['match'].apply(lambda x: difflib.get_close_matches(x, enroll_df['match'], n=1, cutoff=0.8))
enroll_df['mat2'] = high_school_df['match'].apply(lambda x: difflib.get_close_matches(x, enroll_df['match'], n=1, cutoff=0.8))

enroll_df['mat2'] = enroll_df['mat2'].to_string()
high_school_df['mat2'] = high_school_df['mat2'].to_string()
area_sqft_df['mat2'] = area_sqft_df['mat2'].to_string()
temp_df = high_school_df.merge(area_sqft_df, on='mat2', how = 'outer')
# new_df = temp_df.merge(enroll_df, on='mat2',how='right')

temp_df.head()

#### Merge seems to work
## Output a csv to see what is going on
# temp_df.to_csv('new_df.csv')


## cAN'T make the merge work yet
# outputing the dataframes to csv to see what is going on
# high_school_df.to_csv('high_school_df.csv')
# enroll_df.to_csv('enroll_df.csv')
area_sqft_df.to_csv('area_sqft_df.csv')

# ## Merge the dataframes
# new_df = high_school_df.merge(area_sqft_df, on='mat2',how='right')

# new_df.head()
# high_school_df.head()
# enroll_df.head()
# area_sqft_df.head()

In [ ]:
import difflib 

# high_school_df['school_match'] = high_school_df['School'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name']))
# # high_school_df['school_match'] = high_school_df['school_match']
# high_school_df.dtypes

# new_df = high_school_df.join(enroll_df, on='school_name', how='left', lsuffix='_left', rsuffix='_right')
# new_df
enroll_df['school_match'] = enroll_df['school_name']

area_sqft_df['school_match'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_match']))
new_df = area_sqft_df.join(enroll_df, on='school_match', how='left', lsuffix='_left', rsuffix='_right')

# area_sqft_df['school_match'] = area_sqft_df['school_match']
# area_sqft_df.dtypes

high_school_df.dtypes
# area_sqft_df.dtypes


# new_df = area_sqft_df.join(high_school_df, on='school_match', how='left', lsuffix='_left', rsuffix='_right')
# new_df


# # Merge the dataframes
# out_df = pd.merge(high_school_df, area_sqft_df, on='school_match', how='left')
# out_df



# area_sqft_df['school_match'] = area_sqft_df['field'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_match']))
# area_sqft_df
# high_school_df.to_csv('data\high_school_df.csv')

# enroll_df['school_match'] = enroll_df['school_name'].apply(lambda x: difflib.get_close_matches(x, high_school_df['School']))
# enroll_df
# #create duplicate column to retain team name from df2
# high_school_df['team'] = high_school_df['School']

# #convert team name in df2 to team name it most closely matches in df1
# high_school_df['school_name'] = high_school_df['team'].apply(lambda x: difflib.get_close_matches(x, enroll_df['school_name']))

# #merge the DataFrames into one
# out_df = pd.merge(high_school_df, enroll_df, on='school_name', how='left')

# #view final DataFrame
# print(out_df)